# Tracing & Evaluation for Groq based Agents using Maxim AI

Learn how to integrate Maxim observability with the Groq SDK. 

When you use Groq with Maxim instrumentation, the following information is automatically captured for each API call:
- Request Details: Model name, temperature, max tokens, and all other parameters
- Message History: Complete conversation context including system and user messages
- Response Content: Full assistant responses and metadata
- Usage Statistics: Input tokens, output tokens, total tokens consumed
- Cost Tracking: Estimated costs based on Groq’s pricing
- Error Handling: Any API errors or failures with detailed context
- Node Level Evaluations
- Get Real Time Alerts (Slack, PagerDuty, etc.)

[Check Docs](https://getmax.im/HIF14Di)

## Install dependencies & Set environment variables

In [ ]:
# Install dependencies

'''
%pip install groq
%pip install maxim-py
'''

# Set environment variables
import os
os.environ["MAXIM_API_KEY"] = "YOUR_MAXIM_API_KEY"
os.environ["MAXIM_LOG_REPO_ID"] = "YOUR_MAXIM_LOG_REPO_ID"
os.environ["GROQ_API_KEY"] = "YOUR_GROQ_API_KEY"

## Import the required dependencies

In [ ]:
from groq import Groq

from maxim import Config, Maxim
from maxim.logger import LoggerConfig

maxim = Maxim(Config(api_key=os.getenv("MAXIM_API_KEY")))
logger = maxim.logger(LoggerConfig(id=os.getenv("MAXIM_LOG_REPO_ID")))


## Configure Groq with Maxim

In [ ]:
from maxim.logger.groq import instrument_groq

instrument_groq(logger)
client = Groq()

## Simple Inference

In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is a fast LLM useful for?"}
    ],
    model="llama-3.3-70b-versatile"
)

print(chat_completion.choices[0].message.content)


## Define the Tool Calls

In [ ]:
import yfinance as yf

def get_stock_info(symbol: str, key: str):
    data = yf.Ticker(symbol)
    return data.info.get(key, f"Key '{key}' not found for symbol '{symbol}'")

def get_historical_price(symbol: str, start_date: str, end_date: str):
    hist = yf.Ticker(symbol).history(start=start_date, end=end_date).reset_index()
    hist[symbol] = hist['Close']
    return hist[['Date', symbol]].to_dict(orient='records')

def plot_stock_price(data: list, symbol: str, title: str = None):
    """Plot stock price data using plotly"""
    if isinstance(data, str):  # Error message
        print(f"Cannot plot: {data}")
        return None

    df = pd.DataFrame(data)
    df['Date'] = pd.to_datetime(df['Date'])

    if title is None:
        title = f"{symbol} Stock Price Over Time"

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df['Date'],
        y=df[symbol],
        mode='lines',
        name=f'{symbol} Price',
        line=dict(width=2)
    ))

    fig.update_layout(
        title=title,
        xaxis_title="Date",
        yaxis_title="Price ($)",
        hovermode='x unified',
        template='plotly_white'
    )

    fig.show()
    return fig

def compare_stocks(symbols: list, start_date: str, end_date: str):
    """Compare multiple stocks on the same chart"""
    fig = go.Figure()

    for symbol in symbols:
        data = get_historical_price(symbol, start_date, end_date)
        if isinstance(data, str):  # Error message
            print(f"Skipping {symbol}: {data}")
            continue

        df = pd.DataFrame(data)
        df['Date'] = pd.to_datetime(df['Date'])

        fig.add_trace(go.Scatter(
            x=df['Date'],
            y=df[symbol],
            mode='lines',
            name=f'{symbol}',
            line=dict(width=2)
        ))

    fig.update_layout(
        title=f"Stock Price Comparison: {', '.join(symbols)}",
        xaxis_title="Date",
        yaxis_title="Price ($)",
        hovermode='x unified',
        template='plotly_white'
    )

    fig.show()
    return fig


In [ ]:
functions = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_info",
            "description": "Retrieve specific info about a stock",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {"type": "string", "description": "Stock ticker like AAPL or GOOGL"},
                    "key": {"type": "string", "description": "The financial attribute to retrieve (e.g., 'marketCap', 'beta', 'currentPrice')"}
                },
                "required": ["symbol", "key"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_historical_price",
            "description": "Retrieve historical stock price data",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {"type": "string"},
                    "start_date": {"type": "string", "format": "date"},
                    "end_date": {"type": "string", "format": "date"}
                },
                "required": ["symbol", "start_date", "end_date"]
            }
        }
    }
]

## Making LLM Calls with Groq, with tool calling capabilities

In [ ]:
import json


def execute_function_call(function_name: str, arguments: dict):
    """Execute the appropriate function based on the function call"""
    if function_name == "get_stock_info":
        return get_stock_info(**arguments)
    elif function_name == "get_historical_price":
        return get_historical_price(**arguments)
    else:
        return f"Unknown function: {function_name}"

def process_stock_query(query: str, plot_chart: bool = True):
    """Process a stock query and optionally plot results"""

    # Get initial response from Groq
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": "You are a financial assistant. Use the available tools to get stock information and provide helpful analysis."},
            {"role": "user", "content": query}
        ],
        tools=functions,
    )

    messages = [
        {"role": "system", "content": "You are a financial assistant. Use the available tools to get stock information and provide helpful analysis."},
        {"role": "user", "content": query}
    ]

    # Process tool calls if any
    if response.choices[0].message.tool_calls:
        messages.append(response.choices[0].message)

        for tool_call in response.choices[0].message.tool_calls:
            function_name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)

            print(f"Calling function: {function_name} with arguments: {arguments}")

            # Execute the function
            function_result = execute_function_call(function_name, arguments)

            # Add function result to messages
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(function_result)
            })

            # If it's historical price data and plotting is requested, create a chart
            if function_name == "get_historical_price" and plot_chart and not isinstance(function_result, str):
                symbol = arguments.get('symbol', 'Unknown')
                plot_stock_price(function_result, symbol)

        # Get final response with function results
        final_response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=messages,
            tools=functions,
        )

        return final_response.choices[0].message.content
    else:
        return response.choices[0].message.content


## Check Responses on Maxim Dashboard

In [ ]:
print("=== Stock Info Query ===")
result1 = process_stock_query("What is the beta of Meta stock?", plot_chart=False)
print(result1)

In [ ]:
print("=== Historical Price with Chart ===")
result2 = process_stock_query("Show me Apple's stock price for the last 6 months", plot_chart=True)
print(result2)

![Maxim Dashboard](images/maxim_groq.gif)

## Complete Code

In [ ]:
# -*- coding: utf-8 -*-
"""Enhanced Stock Market Function Calling with Plotting"""

import os
import json
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
import yfinance as yf
from groq import Groq
from maxim import Config, Maxim
from maxim.logger import LoggerConfig
from maxim.logger.groq import instrument_groq

# Initialize Maxim and Groq (assuming environment variables are set)
maxim = Maxim(Config(api_key=os.getenv("MAXIM_API_KEY")))
logger = maxim.logger(LoggerConfig(id=os.getenv("MAXIM_LOG_REPO_ID")))

instrument_groq(logger)
client = Groq()

# Date parsing function
def parse_relative_date(date_str: str) -> str:
    """Convert relative date strings to YYYY-MM-DD format"""
    date_str = date_str.lower().strip()
    today = datetime.now()

    if date_str in ['today', 'now']:
        return today.strftime('%Y-%m-%d')
    elif date_str == 'yesterday':
        return (today - timedelta(days=1)).strftime('%Y-%m-%d')
    elif 'month' in date_str:
        # Extract number of months
        import re
        numbers = re.findall(r'\d+', date_str)
        months = int(numbers[0]) if numbers else 1
        # Approximate months as 30 days each
        return (today - timedelta(days=months * 30)).strftime('%Y-%m-%d')
    elif 'week' in date_str:
        numbers = re.findall(r'\d+', date_str)
        weeks = int(numbers[0]) if numbers else 1
        return (today - timedelta(weeks=weeks)).strftime('%Y-%m-%d')
    elif 'day' in date_str:
        numbers = re.findall(r'\d+', date_str)
        days = int(numbers[0]) if numbers else 1
        return (today - timedelta(days=days)).strftime('%Y-%m-%d')
    elif 'year' in date_str:
        numbers = re.findall(r'\d+', date_str)
        years = int(numbers[0]) if numbers else 1
        return (today - timedelta(days=years * 365)).strftime('%Y-%m-%d')
    else:
        # Try to parse as regular date, if it fails return as-is
        try:
            parsed_date = datetime.strptime(date_str, '%Y-%m-%d')
            return date_str
        except:
            # If all else fails, assume it's today
            return today.strftime('%Y-%m-%d')

# Your existing functions with date parsing
def get_stock_info(symbol: str, key: str):
    """Retrieve specific info about a stock"""
    try:
        data = yf.Ticker(symbol)
        return data.info.get(key, f"Key '{key}' not found for symbol '{symbol}'")
    except Exception as e:
        return f"Error retrieving data for {symbol}: {str(e)}"

def get_historical_price(symbol: str, start_date: str, end_date: str):
    """Retrieve historical stock price data"""
    try:
        # Parse relative dates
        parsed_start = parse_relative_date(start_date)
        parsed_end = parse_relative_date(end_date)

        print(f"Parsed dates: {start_date} -> {parsed_start}, {end_date} -> {parsed_end}")

        hist = yf.Ticker(symbol).history(start=parsed_start, end=parsed_end).reset_index()
        hist[symbol] = hist['Close']
        return hist[['Date', symbol]].to_dict(orient='records')
    except Exception as e:
        return f"Error retrieving historical data: {str(e)}"

def plot_stock_price(data: list, symbol: str, title: str = None):
    """Plot stock price data using plotly"""
    if isinstance(data, str):  # Error message
        print(f"Cannot plot: {data}")
        return None

    df = pd.DataFrame(data)
    df['Date'] = pd.to_datetime(df['Date'])

    if title is None:
        title = f"{symbol} Stock Price Over Time"

    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df['Date'],
        y=df[symbol],
        mode='lines',
        name=f'{symbol} Price',
        line=dict(width=2)
    ))

    fig.update_layout(
        title=title,
        xaxis_title="Date",
        yaxis_title="Price ($)",
        hovermode='x unified',
        template='plotly_white'
    )

    fig.show()
    return fig

def compare_stocks(symbols: list, start_date: str, end_date: str):
    """Compare multiple stocks on the same chart"""
    fig = go.Figure()

    for symbol in symbols:
        data = get_historical_price(symbol, start_date, end_date)
        if isinstance(data, str):  # Error message
            print(f"Skipping {symbol}: {data}")
            continue

        df = pd.DataFrame(data)
        df['Date'] = pd.to_datetime(df['Date'])

        fig.add_trace(go.Scatter(
            x=df['Date'],
            y=df[symbol],
            mode='lines',
            name=f'{symbol}',
            line=dict(width=2)
        ))

    fig.update_layout(
        title=f"Stock Price Comparison: {', '.join(symbols)}",
        xaxis_title="Date",
        yaxis_title="Price ($)",
        hovermode='x unified',
        template='plotly_white'
    )

    fig.show()
    return fig

# Function definitions for Groq
functions = [
    {
        "type": "function",
        "function": {
            "name": "get_stock_info",
            "description": "Retrieve specific info about a stock",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {"type": "string", "description": "Stock ticker like AAPL or GOOGL"},
                    "key": {"type": "string", "description": "The financial attribute to retrieve (e.g., 'marketCap', 'beta', 'currentPrice')"}
                },
                "required": ["symbol", "key"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_historical_price",
            "description": "Retrieve historical stock price data. Accepts both absolute dates (YYYY-MM-DD) and relative dates (like '6 months ago', 'today', '1 year ago', etc.)",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {"type": "string", "description": "Stock ticker symbol"},
                    "start_date": {"type": "string", "description": "Start date in YYYY-MM-DD format OR relative date like '6 months ago', '1 year ago'"},
                    "end_date": {"type": "string", "description": "End date in YYYY-MM-DD format OR relative date like 'today', 'yesterday'"}
                },
                "required": ["symbol", "start_date", "end_date"]
            }
        }
    }
]

def execute_function_call(function_name: str, arguments: dict):
    """Execute the appropriate function based on the function call"""
    if function_name == "get_stock_info":
        return get_stock_info(**arguments)
    elif function_name == "get_historical_price":
        return get_historical_price(**arguments)
    else:
        return f"Unknown function: {function_name}"

def process_stock_query(query: str, plot_chart: bool = True):
    """Process a stock query and optionally plot results"""

    # Enhanced system message with date handling instructions
    system_message = """You are a financial assistant. Use the available tools to get stock information and provide helpful analysis.

For date parameters in get_historical_price:
- You can use relative dates like: "6 months ago", "1 year ago", "3 weeks ago", "today", "yesterday"
- Or absolute dates in YYYY-MM-DD format
- The function will automatically parse relative dates to the correct format

Be helpful and provide insightful analysis of the stock data you retrieve."""

    # Get initial response from Groq
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": query}
        ],
        tools=functions,
    )

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": query}
    ]

    # Process tool calls if any
    if response.choices[0].message.tool_calls:
        messages.append(response.choices[0].message)

        for tool_call in response.choices[0].message.tool_calls:
            function_name = tool_call.function.name
            arguments = json.loads(tool_call.function.arguments)

            print(f"Calling function: {function_name} with arguments: {arguments}")

            # Execute the function
            function_result = execute_function_call(function_name, arguments)

            # Add function result to messages
            messages.append({
                "role": "tool",
                "tool_call_id": tool_call.id,
                "content": str(function_result)
            })

            # If it's historical price data and plotting is requested, create a chart
            if function_name == "get_historical_price" and plot_chart and not isinstance(function_result, str):
                symbol = arguments.get('symbol', 'Unknown')
                plot_stock_price(function_result, symbol)

        # Get final response with function results
        final_response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=messages,
            tools=functions,
        )

        return final_response.choices[0].message.content
    else:
        return response.choices[0].message.content

# Example usage
if __name__ == "__main__":
    # Test the enhanced functionality

    # 1. Get stock info (like your original query)
    print("=== Stock Info Query ===")
    result1 = process_stock_query("What is the beta of Meta stock?", plot_chart=False)
    print(result1)
    print()

    # 2. Get historical data and plot
    print("=== Historical Price with Chart ===")
    result2 = process_stock_query("Show me Apple's stock price for the last 6 months", plot_chart=True)
    print(result2)
    print()

    # 3. Manual comparison chart
    print("=== Stock Comparison Chart ===")
    end_date = datetime.now().strftime('%Y-%m-%d')
    start_date = (datetime.now() - timedelta(days=180)).strftime('%Y-%m-%d')

    compare_stocks(['AAPL', 'GOOGL', 'MSFT', 'META'], start_date, end_date)

    # 4. More complex query
    print("=== Complex Analysis ===")
    result3 = process_stock_query(
        "Get Tesla's stock price data for the last 3 months and tell me about its recent performance",
        plot_chart=True
    )
    print(result3)